# Overview

This example notebook will demonstrate Howso Engine’s ability to use case MDA to identify centroids in the data, then use the influential cases of each centroid to conduct matching of the data. 

Definitions:
 
    - case_mda: mean decrease in accuracy for the local model as if each individual case were not in the model 

    - influential_cases: returns the most influential cases and their influence weights based on generalized distances as probability mass 

Note: case_mda is available for "targeted" workflow only, influential_cases are available for both "targeted" and "targetless" workflows. 

In [1]:
from pprint import pprint
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots
from scipy.special import (
    boxcox,
    inv_boxcox,
)
from sklearn.datasets import make_blobs
from sklearn.metrics import (
    accuracy_score,
    pairwise_distances,
)

from howso import engine
from howso.utilities import infer_feature_attributes

init_notebook_mode(connected=True)

# Generate a simple dataset

For this example, we will create a simple data set which consists of three features. 


In [2]:
# Generate simple data set with three features
blobs = make_blobs(n_samples=1000, n_features=3, centers=50, random_state=0)
df = pd.DataFrame(blobs[0], columns=['x', 'y', 'price'])

#Rescale price feature such that it is always positive
df['price'] = df['price'] + abs(df['price'].min()) + 1

# # (Optional) Inverse Box-Cox transform to create a Pareto-like distribution
# df['x'] = inv_boxcox(df['x'], .5)
# df['price'] = inv_boxcox(df['price'], .25)

# Store the target cluster for demonstration evaluation
target = blobs[1]
df.shape

(1000, 3)

Visualization of the dataset to show the clusters. Note, the plot below is only showing two dimensions.

In [3]:
fig = px.scatter(
    df, x="x", y="y",
    color=pd.Series(target, dtype=str, name="Center"),
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=650, height=650,
    render_mode='webgl',
    opacity=0.5,
)
iplot(fig)

In [4]:
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Histogram(x=df["x"], name="X"), row=1, col=1)
fig.add_trace(go.Histogram(x=df["price"], name="Price"), row=1, col=2, )
fig.update_layout(dict(width=1300, height=650, title="Histograms for 'X' and 'Price'"))
iplot(fig)

# Build Howso Engine model

In this section, we will build and analyze the Howso Engine model.

In [5]:
# Identify feature types
features = infer_feature_attributes(df)

# Instantiate and train the model 
t = engine.Trainee(name="Engine - Fuzzy Matching Recipe",  features=features, overwrite_existing=True)
t.train(cases=df)

# Specify the context (independent variable) and action (target) feature
context_features = features.get_names(without=['price'])
action_features = 'price'

# Analyze the model
t.analyze(context_features=context_features, action_features=[action_features])

# Store the session ID to retrieve training cases for later evaluation
sessions = t.get_sessions()
session_id = sessions[0]['id']
session_id

The following parameters from configuration file will override the Amalgam parameters set in the code: {'trace'}


'620239b3-5843-4989-a8db-fa6fd151db4b'

In [6]:
# Feature types
pprint(features)

{'price': {'bounds': {'max': 54.598150033144236, 'min': 1.0},
           'decimal_places': 16,
           'original_type': {'data_type': 'numeric', 'size': 8},
           'type': 'continuous'},
 'x': {'bounds': {'max': 20.085536923187668, 'min': -20.085536923187668},
       'decimal_places': 19,
       'original_type': {'data_type': 'numeric', 'size': 8},
       'type': 'continuous'},
 'y': {'bounds': {'max': 20.085536923187668, 'min': -20.085536923187668},
       'decimal_places': 18,
       'original_type': {'data_type': 'numeric', 'size': 8},
       'type': 'continuous'}}


In [7]:
# Hyperparameters for the model
pprint(t.get_params())

{'analyze_growth_factor': 7.389056,
 'analyze_threshold': 100,
 'auto_analyze': False,
 'auto_analyze_limit_size': 200000,
 'default_hyperparameter_map': {'allFeatureResidualsCached': False,
                                'dt': -1,
                                'featureDeviations': None,
                                'featureDomainAttributes': {'price': 53.598150033144236,
                                                            'x': 40.171073846375336,
                                                            'y': 40.171073846375336},
                                'featureWeights': None,
                                'k': 8,
                                'p': 0.1,
                                'paramPath': ['.default'],
                                'useDeviations': False},
 'hyperparameter_map': {'price': {'x.y.': {'full': {'.none': {'allFeatureResidualsCached': False,
                                                              'dt': -1,
                        

In [8]:
# Get a list of training cases
case_inds = t.get_cases(session=session_id, features=['.session', '.session_training_index']).values

# Identify centroids - case_mda

Identify the case_mda for each of the training cases. The case_mda will be used as centroids during clustering.

In [9]:
# React to each case, extract the case_mda
results = t.react(case_indices=case_inds, preserve_feature_values=context_features,
                  action_features=[action_features], 
                  leave_case_out=True,
                  details={'case_mda': True, 'robust_influences': True})

In [10]:
# Combine the case_mda into a single DataFrame
case_mdas = pd.DataFrame()

for c in results['details']['case_mda']:
    case_mdas = pd.concat([case_mdas, pd.DataFrame(c)])

In [11]:
# Rank the mda and then find the top fifty centroids. Note, the number of centroids can be tuned. 
num_centers = 50
case_mdas = case_mdas.sort_values('mda', ascending=False).reset_index(drop=True)
case_mdas = case_mdas[~case_mdas['.session_training_index'].duplicated()]
centers = case_mdas[0:num_centers]

In [12]:
# The .session and .session_training_index will be used to find the influential cases in the next section
centers.head()

,.session_training_index,.session,mda
0,540,620239b3-5843-4989-a8db-fa6fd151db4b,0.880911
1,941,620239b3-5843-4989-a8db-fa6fd151db4b,0.873796
2,330,620239b3-5843-4989-a8db-fa6fd151db4b,0.869878
4,173,620239b3-5843-4989-a8db-fa6fd151db4b,0.827218
7,714,620239b3-5843-4989-a8db-fa6fd151db4b,0.804049


In [13]:
# Get the case values for visualization
center_cases = t.get_cases(case_indices=centers[['.session', '.session_training_index']].values)
center_cases.head()

,x,y,price
0,-1.106303,7.516132,21.770924
1,-1.159834,7.524574,21.176786
2,-0.807051,10.014079,3.221469
3,6.663419,8.698265,24.074721
4,8.719175,3.859010,23.791220


In [14]:
fig = px.scatter(
    df, x="x", y="y",
    color=pd.Series(target, dtype=str, name="Center"),
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=650, height=650,
    render_mode='webgl',
    opacity=0.1,
)
fig.add_trace(go.Scattergl(x=center_cases["x"], y=center_cases["y"], mode="markers"))

iplot(fig)

Two dimensional representation of the identified centroids (dark blue). 

# Cluster around centroids - using influential cases

Use the influential cases of each centroid to cluster the data. More specifically, tag the influential cases as the same cluster as the centroid.

Note: most similar cases is an alternative method to cluster data points. 

In [15]:
# Get the influential cases and most similar cases for each of the centroids
num_similar_cases = 5
results = t.react(case_indices=centers[[".session", ".session_training_index"]].values,
                  leave_case_out=True,
                  preserve_feature_values=context_features, action_features=[action_features], 
                  details={'most_similar_cases':True, 'num_most_similar_cases':num_similar_cases, 'influential_cases':True})

In [16]:
# Tag each influential cases with the same cluster as the centroid
centers = centers.reset_index().rename(columns={'index':'target'})
clustered_df = pd.DataFrame()

for i, cases in enumerate(results['details']['influential_cases']):
    tar = centers['target'].iloc[i]
    clustered = pd.DataFrame(cases)
    clustered['target'] = tar

    clustered_df = pd.concat([clustered_df, clustered])
    
clustered_df = clustered_df.reset_index(drop=True)

In [17]:
# Training data for evaluation purposes
df_train = t.get_cases(session=session_id, features=df.columns.tolist() + ['.session_training_index'])
df_train = df_train.join(pd.Series(target, name='target'))

df_train

,x,y,price,.session_training_index,target
0,-3.235779,-1.660880,20.497487,0,47
1,4.283446,6.814812,21.116852,1,6
2,-3.011843,-2.788841,8.100008,2,19
3,-3.863634,-1.995433,14.314644,3,16
4,1.937476,-9.014279,21.314027,4,32
...,...,...,...,...,...
995,-1.755072,7.239146,21.026818,995,2
996,-0.850211,10.007344,4.005985,996,17
997,1.363724,3.038060,19.619870,997,12
998,-8.880399,-4.024742,4.517326,998,25


In [18]:
clustered_df

,.session_training_index,.session,price,.influence_weight,x,y,target
0,941,620239b3-5843-4989-a8db-fa6fd151db4b,21.176786,0.514019,-1.159834,7.524574,0
1,136,620239b3-5843-4989-a8db-fa6fd151db4b,19.403382,0.072031,-1.546047,7.526447,0
2,377,620239b3-5843-4989-a8db-fa6fd151db4b,21.449441,0.060965,-0.665302,7.436011,0
3,640,620239b3-5843-4989-a8db-fa6fd151db4b,19.048701,0.055443,-0.754514,7.719356,0
4,866,620239b3-5843-4989-a8db-fa6fd151db4b,20.688432,0.051370,-0.858162,7.183051,0
...,...,...,...,...,...,...,...
645,789,620239b3-5843-4989-a8db-fa6fd151db4b,4.054625,0.060897,7.023611,2.499241,109
646,990,620239b3-5843-4989-a8db-fa6fd151db4b,4.377537,0.060754,7.216219,3.070217,109
647,859,620239b3-5843-4989-a8db-fa6fd151db4b,2.529861,0.060639,5.337411,2.901299,109
648,465,620239b3-5843-4989-a8db-fa6fd151db4b,20.858177,0.056318,6.565573,3.706117,109


In [19]:
# Map each of the predicted cluster with the most commonly occurring original cluster
label_map = {}

for c in clustered_df['target'].unique():
    group_df = clustered_df[clustered_df['target'] == c]
    target_original = df_train[df_train['.session_training_index'].isin(group_df['.session_training_index'])]['target'].value_counts().idxmax()
    # print(df_train[df_train['.session_training_index'].isin(group_df['.session_training_index'])]['target'].value_counts())
    label_map[c] = target_original
    
clustered_df = clustered_df.replace({'target':label_map})

In [20]:
# Create a single table with predicted and correct target results
correct_target = []
for i in clustered_df['.session_training_index']:
    correct_t = df_train[df_train['.session_training_index'] == i]['target'].iloc[0]
    correct_target.append(correct_t)
    
clustered_df['correct_target'] = correct_target
clustered_df.head()

,.session_training_index,.session,price,.influence_weight,x,y,target,correct_target
0,941,620239b3-5843-4989-a8db-fa6fd151db4b,21.176786,0.514019,-1.159834,7.524574,2,2
1,136,620239b3-5843-4989-a8db-fa6fd151db4b,19.403382,0.072031,-1.546047,7.526447,2,2
2,377,620239b3-5843-4989-a8db-fa6fd151db4b,21.449441,0.060965,-0.665302,7.436011,2,2
3,640,620239b3-5843-4989-a8db-fa6fd151db4b,19.048701,0.055443,-0.754514,7.719356,2,2
4,866,620239b3-5843-4989-a8db-fa6fd151db4b,20.688432,0.051370,-0.858162,7.183051,2,2


In [21]:
# Accuracy evaluation
acc = round(accuracy_score(clustered_df['correct_target'], clustered_df['target']), 3)
print(f'accuracy: {acc}')

accuracy: 0.6


Two dimensional representation of the identified clusters.

In [22]:
fig = px.scatter(
    df, x="x", y="y",
    color=pd.Series(target, dtype=str, name="Center"),
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=650, height=650,
    render_mode='webgl',
    opacity=0.1,
)
for group, data in clustered_df.groupby("target"):
    fig.add_trace(go.Scattergl(
        x=data["x"],
        y=data["y"],
        mode="markers",
        opacity=0.5
    ))

iplot(fig)

# Impact of number of centroids

Howso Engine starts clustering with the most confident centroids (highest case_mda). Therefore, the number specified centroids can have an impact on clustering performance. 

Relatively low ratio of specified clusters vs. actual number of clusters is expected to have higher accuracy. In this example, we will demonstrate the impact of the number of cluster using 10 to 50 clusters. As a ratio of specified clusters to actual clusters, these represents a ratio of 1/5 up to 1. 

In [23]:
# Number of specified clusters
num_centers = [10, 20, 30, 40, 50]

In [24]:
# A function which finds the centroids (using case_mda), then clusters the data (using influential cases), followed by accuracy evaluation
def cluster_eval(num_centers):
    case_mdas = pd.DataFrame()
    
    results = t.react(case_indices=case_inds, preserve_feature_values=context_features,
                  action_features=[action_features], 
                  leave_case_out=True,
                  details={'case_mda':True, 'robust_influences': True})

    for c in results['details']['case_mda']:
        case_mdas = pd.concat([case_mdas, pd.DataFrame(c)])

    case_mdas = case_mdas.sort_values('mda', ascending=False).reset_index(drop=True)
    case_mdas = case_mdas[~case_mdas['.session_training_index'].duplicated()]
    centers = case_mdas[0:num_centers]

    center_cases = t.get_cases(case_indices=centers[['.session', '.session_training_index']].values)

    num_similar_cases = 5

    results = t.react(case_indices=centers[['.session', '.session_training_index']].values,
                      leave_case_out=True,
                      preserve_feature_values=context_features, action_features=[action_features], 
                      details={'most_similar_cases':True, 'num_most_similar_cases':num_similar_cases, 'influential_cases':True})

    centers = centers.reset_index().rename(columns={'index':'target'})
    clustered_df = pd.DataFrame()

    for i, cases in enumerate(results['details']['influential_cases']):
        tar = centers['target'].iloc[i]
        clustered = pd.DataFrame(cases)
        clustered['target'] = tar

        clustered_df = pd.concat([clustered_df, clustered])

    clustered_df = clustered_df.reset_index(drop=True)

    df_train = t.get_cases(session=session_id, features=df.columns.tolist() + ['.session_training_index'])
    df_train = df_train.join(pd.Series(target, name='target'))

    label_map = {}

    for c in clustered_df['target'].unique():
        group_df = clustered_df[clustered_df['target'] == c]
        target_original = df_train[df_train['.session_training_index'].isin(group_df['.session_training_index'])]['target'].value_counts().idxmax()
        # print(df_train[df_train['.session_training_index'].isin(group_df['.session_training_index'])]['target'].value_counts())
        label_map[c] = target_original

    clustered_df = clustered_df.replace({'target':label_map})

    correct_target = []
    for i in  clustered_df['.session_training_index']:
        correct_t = df_train[df_train['.session_training_index'] == i]['target'].iloc[0]
        correct_target.append(correct_t)

    clustered_df['correct_target'] = correct_target

    clustered_df.head()

    return(accuracy_score(clustered_df['correct_target'], clustered_df['target']))

In [25]:
# Get the accuracy results for each of specified number of clusters
acc = []
for n in num_centers:
    acc.append(cluster_eval(n))

Visualize the accuracy vs. number of specified centroid results

In [26]:
fig = px.line(x=num_centers, y=acc, width=650, height=650)
fig.update_layout(xaxis_title="number of centroids", yaxis_title="accuracy")
iplot(fig)